<a href="https://colab.research.google.com/github/stephenbeckr/randomized-algorithm-class/blob/master/Demos/demo21_randomizedSVDs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Randomized SVDs

Comparing various randomized SVDs, namely:

1. "randomized Simultaneous Iteration" aka randomized Subspace Iteration aka randomized Orthogonal Iteration.
  - see [Rokhlin, Szlam, Tygert, 2009](https://arxiv.org/abs/0809.2274)
  - also in [Halko, Martinsson, Tropp, 2011, SIAM Review](https://epubs.siam.org/doi/10.1137/090771806)
2. "randomized block Krylov iteration"
  - for analysis, see [Musco, Musco, 2015](http://arxiv.org/abs/1504.05477)

We don't do very fancy comparisons, just a very quick check. Implementations have not been tested extensively.

Code by Stephen Becker, Oct 2021

In [46]:
import numpy as np
import numpy.linalg
import scipy.linalg
import scipy.sparse.linalg
from numpy.linalg import norm
from numpy.random import default_rng
rng = default_rng()
from matplotlib import pyplot as plt

Define the `bksvd` (randomized block Krylov SVD) and `sisvd` (randomized Subspace Iteration SVD), based on implmentation at [bksvd](https://github.com/cpmusco/bksvd) which is from the Musco and Musco 2015 paper

In [30]:
import numpy as np
import numpy.linalg
QR = lambda A : np.linalg.qr(A,mode='reduced')[0]

def bksvd(A,k,iter=3,bsize=None, rng = np.random.default_rng() ):
  """
  Randomized block Krylov iteration for truncated singular value decomposition
  Computes approximate top singular vectors and corresponding values
  Described in Musco, Musco, 2015 (http://arxiv.org/abs/1504.05477)
  input:
    * A : matrix to decompose, size m x n
        (either a 2D numpy array or of type scipy.linalg.sparse.LinearOperator)
    * k : number of singular vectors to compute, default = 6
    * iter : number of iterations, default = 3
    * bsize : block size, must be >= k, default = k
    * rng : [optional] random number generator from np.random
  output:
    k singular vector/value pairs. 
    * U : a matrix whose columns are approximate top left singular vectors for A
    * S : a diagonal matrix whose entries are A's approximate top singular values
    * Vh : a matrix whose rows are approximate top right singular vectors for A
    [Note: Vh is V.T, following np.linalg.svd convention; this is the transpose
     of the Matlab svd convention ]

    U*S*V' is a near optimal rank-k approximation for A

    based on Christopher Musco's Matlab code at
    https://github.com/cpmusco/bksvd/blob/master/bksvd.m 

    TODO: (efficiently) transpose if m << n
  """
  m,n = A.shape
  if k > m or k > n:
    raise ValueError("k must be smaller than m,n")
  iter = int(iter)
  if iter < 1:
    raise ValueError("iter must be 1 or larger")
  if bsize is None:
    bsize = k
  bsize = int(bsize)
  if bsize < k:
    raise ValueError("bsize must be at least k")
  
  Y = np.zeros( (n,bsize*iter) )

  Omega   = rng.standard_normal( (n,bsize) )
  Omega   = QR(Omega)

  for i in range(iter):
    Omega = A.T@( A@Omega )
    Omega = QR(Omega)
    Y[:,i*bsize:(i+1)*bsize] = Omega
  
  # Now do big QR (*not* attemtping Lanczos trick)
  Q   = QR(Y)
  T   = A@Q

  Ut,st,Vt_h = np.linalg.svd( T, full_matrices=False, compute_uv=True)
  s   = st[:k]
  U   = Ut[:,:k]
  Vh  = Vt_h[:k,:]@Q.T
  return U, s, Vh


def sisvd(A,k,iter=3,bsize=None, rng = np.random.default_rng() ):
  """
  Simple randomized Simultaneous Iteration for truncated SVD
  Computes approximate top singular vectors and corresponding values
  Described in Rokhlin, Szlam, Tygert, 2009 (https://arxiv.org/abs/0809.2274)
    and nice Matlab code in:
    Huamin Li, George C. Linderman, Arthur Szlam, Kelly P. Stanton, Yuval Kluger, 
    and Mark Tygert. 
    "Algorithm 971: An Implementation of a Randomized Algorithm for Principal 
    Component Analysis". ACM Trans. Math. Softw. 43, 3, 2017
    DOI: https://doi.org/10.1145/3004053


  input:
    * A : matrix to decompose, size m x n
        (either a 2D numpy array or of type scipy.linalg.sparse.LinearOperator)
    * k : number of singular vectors to compute, default = 6
    * iter : number of iterations, default = 3
    * bsize : block size, must be >= k, default = k
    * rng : [optional] random number generator from np.random
  output:
    k singular vector/value pairs. 
    * U : a matrix whose columns are approximate top left singular vectors for A
    * S : a diagonal matrix whose entries are A's approximate top singular values
    * Vh : a matrix whose rows are approximate top right singular vectors for A
    [Note: Vh is V.T, following np.linalg.svd convention; this is the transpose
     of the Matlab svd convention ]

    U*S*V' is a near optimal rank-k approximation for A

    based on Christopher Musco's Matlab code at
    https://github.com/cpmusco/bksvd/blob/master/sisvd.m

    TODO: (efficiently) transpose if m << n
  """
  m,n = A.shape
  if k > m or k > n:
    raise ValueError("k must be smaller than m,n")
  iter = int(iter)
  if iter < 1:
    raise ValueError("iter must be 1 or larger")
  if bsize is None:
    bsize = k
  bsize = int(bsize)
  if bsize < k:
    raise ValueError("bsize must be at least k")
  
  Y = np.zeros( (n,bsize*iter) )

  Omega   = rng.standard_normal( (n,bsize) )
  Omega   = QR(Omega)

  for i in range(iter):
    Omega = A.T@( A@Omega )
    Omega = QR(Omega)
  
  T   = A@Omega

  Ut,st,Vt_h = np.linalg.svd( T, full_matrices=False, compute_uv=True)
  s   = st[:k]
  U   = Ut[:,:k]
  Vh  = Vt_h[:k,:]@Omega.T
  return U, s, Vh

### Some test matrices to find SVD of

In [41]:
M, N = int(8e3), int(5e3)
# M, N = 50, 50  # for error checking

A   = rng.standard_normal( (M,N) )@np.diag(np.logspace(0,3,N))@(
    rng.standard_normal((N,N) ) + 0.1*np.eye(N) )

Use a standard dense SVD for accuracy baseline

In [42]:
%time U,s,Vh = np.linalg.svd( A, full_matrices=False)

CPU times: user 5min 36s, sys: 9.12 s, total: 5min 45s
Wall time: 2min 58s


Compare the 2 randomized methods as well as a standard Lanczos-based iterative method from `scipy`:

In [47]:
k   = 5
q   = 20  # number of subspace/Lanczos iterations

print('-- top k sing. values using classical methods --')
with np.printoptions(precision=2,suppress=True):
  print(s[:k])

print(f'\n-- using randomized block Lanczos --')
%time UU,ss,VVh = bksvd(A,k,iter=q)
with np.printoptions(precision=2,suppress=True):
  print(ss)
  print( UU.T@U[:,:k] )
  print( VVh@Vh.T[:,:k] )

print(f'\n-- using randomized subspace iteration --')
%time UU,ss,VVh = sisvd(A,k,iter=q)
with np.printoptions(precision=2,suppress=True):
  print(ss)
  print( UU.T@U[:,:k] )
  print( VVh@Vh.T[:,:k] )

print(f'\n-- using non-randomized non-block Lanczos --')
%time UU,ss,VVh = scipy.sparse.linalg.svds(A, k )
with np.printoptions(precision=2,suppress=True):
  print(ss)
  print( UU.T@U[:,:k] )
  print( VVh@Vh.T[:,:k] )

-- top k sing. values using classical methods --
[7521984.64 7465124.83 7438533.37 7365519.38 7332619.27]

-- using randomized block Lanczos --
CPU times: user 13.7 s, sys: 1.09 s, total: 14.8 s
Wall time: 7.68 s
[7521984.63 7465124.76 7438532.94 7365324.03 7332578.48]
[[ 1.  0. -0. -0.  0.]
 [ 0. -1.  0. -0.  0.]
 [ 0.  0.  1. -0. -0.]
 [-0.  0. -0. -1. -0.]
 [ 0. -0. -0.  0. -1.]]
[[ 1.  0. -0. -0.  0.]
 [ 0. -1.  0. -0.  0.]
 [ 0.  0.  1. -0. -0.]
 [-0.  0. -0. -1. -0.]
 [ 0. -0. -0.  0. -1.]]

-- using randomized subspace iteration --
CPU times: user 12.2 s, sys: 969 ms, total: 13.2 s
Wall time: 6.73 s
[7506730.69 7411573.78 7400905.07 7295638.22 7182102.18]
[[-0.96 -0.05  0.17 -0.   -0.06]
 [ 0.03  0.64  0.6   0.12  0.27]
 [-0.1   0.54 -0.64  0.05  0.31]
 [-0.02 -0.01 -0.03 -0.66  0.28]
 [ 0.07  0.03  0.11  0.15 -0.19]]
[[-0.96 -0.06  0.18 -0.   -0.06]
 [ 0.03  0.64  0.6   0.12  0.27]
 [-0.1   0.53 -0.64  0.06  0.31]
 [-0.02 -0.01 -0.03 -0.66  0.28]
 [ 0.07  0.02  0.11  0.14 -0.18